# Classificação dos Reviews

Decidi treinar um classificador porque, uma vez ajustado, ele prediz milhares de reviews a custo quase zero e em milissegundos, evitando as chamadas caras e lentas a um LLM para cada exemplo; além disso, apliquei uma estratégia híbrida: começar com rótulos fracos derivados do score para cobrir rapidamente todo o conjunto e depois refinar o modelo com um subconjunto de rótulos “gold” gerados pelo LLM, aproveitando sua alta precisão apenas onde faz diferença

In [ ]:
import sqlalchemy as sa
import polars as pl
from pathlib import Path
from openai import OpenAI
import time
import os
from dotenv import load_dotenv

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import cross_val_predict, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import numpy as np
from sklearn.preprocessing import LabelEncoder

load_dotenv(dotenv_path=Path().resolve().parent / ".env");

Carregando e tratando dataset

In [35]:
DB_PATH = Path().resolve().parent / "data" / "olist.db"
engine = sa.create_engine(f"sqlite:///{DB_PATH}")

review_product = """
    SELECT order_reviews.review_id,
        order_reviews.order_id,
        order_items.product_id,
        order_reviews.review_score,
        order_reviews.review_comment_message AS review_text,
        order_reviews.review_creation_date,
        order_reviews.review_answer_timestamp
    FROM order_reviews
    JOIN order_items ON order_items.order_id = order_reviews.order_id
    WHERE order_reviews.review_comment_message IS NOT NULL
      AND order_reviews.review_comment_message != ''
      AND order_reviews.order_id IN (
          SELECT order_id
          FROM order_items
          GROUP BY order_id
          HAVING COUNT(DISTINCT product_id) = 1
      )
"""

df_review_product = pl.read_database(review_product, engine)

In [36]:
df_review_product = df_review_product.with_columns(
    pl.struct(["order_id", "product_id"])
    .map_elements(lambda s: f"{s['order_id']}_{s['product_id']}", return_dtype=pl.Utf8)
    .alias("doc_id")
).unique(subset=["doc_id"])

## Gerando datasets de Holdout e Treino

In [55]:
import polars as pl
from sklearn.model_selection import train_test_split

train, holdout = train_test_split(
    df_review_product,
    test_size=200,
    stratify=df_review_product["review_score"],
    random_state=42,
)


Confirmando a distribuição dos score reviews nos datasets

In [56]:
(
    holdout.group_by("review_score")
    .len()
    .with_columns(((pl.col("len") / holdout.height) * 100).round(1).alias("proportion"))
    .sort("proportion", descending=True)
)

review_score,len,proportion
i64,u32,f64
5,103,51.5
1,40,20.0
4,30,15.0
3,17,8.5
2,10,5.0


In [57]:
(
    train.group_by("review_score")
    .len()
    .with_columns(((pl.col("len") / train.height) * 100).round(1).alias("proportion"))
    .sort("proportion", descending=True)
)


review_score,len,proportion
i64,u32,f64
5,15536,51.5
1,5990,19.9
4,4537,15.1
3,2590,8.6
2,1489,4.9


Criando os Weak Label para treianr o modelo inicial

In [59]:
train = train.with_columns(
    pl.when(pl.col("review_score").is_in([1, 2]))
    .then(pl.lit("negativo"))
    .when(pl.col("review_score") == 3)
    .then(pl.lit("neutro"))
    .when(pl.col("review_score").is_in([4, 5]))
    .then(pl.lit("positivo"))
    .otherwise(None)
    .alias("sentiment")
)

Criando os labels com LLM para o holdout

In [44]:
def classificar_reviews_openai(client, batch):
    prompt_template = (
        "Classifique cada review abaixo como 'positivo', 'neutro' ou 'negativo'. "
        "Responda no formato: <doc_id>: <classificação>\n\n"
    )
    prompt = prompt_template
    for row in batch.iter_rows(named=True):
        prompt += f"{row['doc_id']}: {row['review_text']}\n"
    prompt += "\nResponda apenas com as classificações."

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    classificacoes = {}
    for line in response.choices[0].message.content.strip().split("\n"):
        if ":" in line:
            doc_id, label = line.split(":", 1)
            classificacoes[doc_id.strip()] = label.strip().lower()
    return classificacoes

In [50]:
holdout_batches = [holdout.slice(i, 10) for i in range(0, holdout.height, 10)]
labels = {}

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

for i, batch in enumerate(holdout_batches):
    print(f"Classificando Batch {i}")
    batch_labels = classificar_reviews_openai(client, batch)
    print(f"Resultado:\n{batch_labels}")
    labels.update(batch_labels)
    time.sleep(1)  # Evita rate limit

Classificando Batch 0
Resultado:
{'bbe56245f9be98b922ff6c35992ba2da_57b5e5e19622c1aaf0fd159f5d9d5b4f': 'positivo', 'e912fe377f77944ec68b3650ead100bf_0fa8853b873ba06e7d27385fbb1411ca': 'neutro', 'fcd0b91ad7d520a891affcc33e293566_63cf4d771cba1d380af927afe5895d4b': 'negativo', '4e7215ac8ad76bd95a10b8230492c690_f961683d82cf9021acfcd05a6c9e38d0': 'positivo', '556bb5c8ecb64be5ee1ee039d4376432_f5d5fa2bc95883494c61ae05a351348a': 'positivo', '774e1d0dad9de39af357f92c7180e21f_a62e25e09e05e6faf31d90c6ec1aa3d1': 'neutro', '504a90faaf31b2457b86f7179a6bd79e_f264c1d9b20b5e4a340254d0405e613b': 'positivo', '3c7299e1335df876b15d9b4cd3858fa9_23c6236434e58c6519c19e56c2dade45': 'positivo', '07dc1dc61c5ee9a1387069fec64b6663_342936541051cf54fbe4879bd6e84985': 'positivo', '2804400973412de4efdaf1399281199c_23bcd6822a33df5534f9b290216eec1f': 'positivo'}
Classificando Batch 1
Resultado:
{'ba714b832c8a26b8b6ce921e0ecbc886_8c292ca193d326152e335d77176746f0': 'positivo', '93355a1eda6104b6fc69e20c6dfa753a_4df79064ddc

In [60]:
holdout = holdout.with_columns(
    pl.col("doc_id")
    .map_elements(lambda x: labels.get(x, None), return_dtype=pl.Utf8)
    .alias("sentiment")
)

In [66]:
# Calcula a crosstab absoluta
crosstab = holdout.group_by(["sentiment", "review_score"]).len().sort("review_score").pivot(
    values="len",
    index="sentiment",
    on="review_score"
).fill_null(0)

crosstab

sentiment,1,2,3,4,5
str,u32,u32,u32,u32,u32
"""negativo""",36,6,7,1,1
"""neutro""",4,2,7,8,8
"""positivo""",0,2,3,21,94


> Podemos perceber que a classificação por score acerta relativamente bem os negativos e positivos mas não os neutros, por isso não podemos usar simplemente o score como informação de predição pro sentimento do review

Salvando os datasets para uso posterior facilitado

In [ ]:
holdout.write_parquet(Path().resolve().parent / "data" / "gold" / "holdout.parquet")
train.write_parquet(Path().resolve().parent / "data" / "silver" / "train.parquet")

## Treinando o modelo apenas com Weak Labels

Extraindo Features (X) e Target (y) e gerando os embeddings que serão usados para previsão

In [72]:
features = train["review_text"].to_list()
target = train["sentiment"].to_list()

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
X = model.encode(features, show_progress_bar=True)
y = np.array(labels)

Batches: 100%|██████████| 942/942 [00:27<00:00, 34.69it/s]


Fazendo o treinamento do modelo usando Cross Validation Estratificada

In [ ]:
clf = XGBClassifier(eval_metric="mlogloss", random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Codifica os labels para inteiros
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Cross Validate
result = cross_val_predict(clf, X, y_encoded, cv=cv, verbose=1)

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   37.2s finished


In [87]:
y_pred = le.inverse_transform(result)

# Avaliação
print(classification_report(y, y_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y, y_pred))


              precision    recall  f1-score   support

    negativo     0.7409    0.8581    0.7952      7479
      neutro     0.2939    0.0614    0.1016      2590
    positivo     0.8989    0.9377    0.9179     20073

    accuracy                         0.8426     30142
   macro avg     0.6446    0.6191    0.6049     30142
weighted avg     0.8077    0.8426    0.8173     30142

Confusion Matrix:
[[ 6418   174   887]
 [ 1202   159  1229]
 [ 1043   208 18822]]


> Como podemos ver, com base no f1-score, as classes positivas e negativas são mais fáceis de prever com base apenas no Score já a neutra é muito mais difícil, por isso vamos agora treinar o modelo e pegar aqueles registros que o modelo encontrou dificuldade de prever

Vamos treinar o weak na base de treino e avaliar a performance na holdout para podermos comparar depois após o treino do modelo em gold label

In [88]:
clf_weak = clf.fit(X, y_encoded)

In [91]:
# Extrai os textos e labels reais do holdout
holdout_texts = holdout["review_text"].to_list()
holdout_y = holdout["sentiment"].to_list()

# Gera embeddings para o holdout
holdout_X = model.encode(holdout_texts, show_progress_bar=True)

# Faz predições
holdout_pred_encoded = clf_weak.predict(holdout_X)
holdout_pred = le.inverse_transform(holdout_pred_encoded)

# Avaliação
print(classification_report(holdout_y, holdout_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(holdout_y, holdout_pred))

Batches: 100%|██████████| 7/7 [00:01<00:00,  6.81it/s]


              precision    recall  f1-score   support

    negativo     0.7925    0.8235    0.8077        51
      neutro     0.6667    0.0690    0.1250        29
    positivo     0.8194    0.9833    0.8939       120

    accuracy                         0.8100       200
   macro avg     0.7595    0.6253    0.6089       200
weighted avg     0.7904    0.8100    0.7605       200

Confusion Matrix:
[[ 42   1   8]
 [  9   2  18]
 [  2   0 118]]


> Da mesma forma que na validaç!ao cruzada, no Holdout, a performance geral de positivo e negativo foi boa mas neutro não

In [98]:
# Fazendo previsões de probabilidade
probas = clf_weak.predict_proba(X)

# Pegando a maior probabilidade e os labels
max_probas = np.max(probas, axis=1)
pred_labels = le.inverse_transform(np.argmax(probas, axis=1))

# Sinaliza registros com baixa certeza (<90%)
pred_final = np.where(max_probas >= 0.8, pred_labels, "incerto")

# Adicionando a predição no dataframe
train = train.with_columns(pl.Series("sentiment_pred", pred_final))

Vamos verificar quantos registros ficaram com proba menor que 80% para o modelo.

In [99]:
train.group_by("sentiment_pred").len().sort("sentiment_pred")

sentiment_pred,len
str,u32
"""incerto""",2514
"""negativo""",6969
"""neutro""",1298
"""positivo""",19361


In [103]:
# Calcula a crosstab absoluta
crosstab = (
    train.group_by(["sentiment_pred", "review_score"])
    .len()
    .sort("review_score")
    .pivot(values="len", index="sentiment_pred", on="review_score")
    .fill_null(0)
)

crosstab


sentiment_pred,1,2,3,4,5
str,u32,u32,u32,u32,u32
"""positivo""",12,18,113,4100,15118
"""negativo""",5668,1290,6,3,2
"""incerto""",310,181,1173,434,416
"""neutro""",0,0,1298,0,0


> Será os registros marcados como incerto que serão classificados pelo LLM para ajudar na predição

### Criando dataset com rotulagem Gold

In [110]:
df_incerto = train.filter(pl.col('sentiment_pred') == "incerto").select(
    pl.all().exclude("sentiment_pred", "sentiment")
)
df_incerto.write_parquet(Path().resolve().parent / "data" / "silver" / "to_refine.parquet")

Lendo o dataset com os registros preditos pelo LLM

In [115]:
train_refined = pl.read_parquet(Path().resolve().parent / "data" / "gold" / "train_refined.parquet")

train_refined.group_by("sentiment").len()

sentiment,len
str,u32
"""neutro""",700
"""positivo""",813
"""negativo""",1001


## Treinando o modelo com os Gold Labels

Vamos mesclar os dados de treino weak com os dados de treino gold e dar pesos diferentes no treino para cada tipo de previsão

In [121]:
train = pl.read_parquet(Path().resolve().parent / "data" / "silver" / "train.parquet")
train = train.with_columns(pl.lit("weak").alias("pred_source"))
train_refined = train_refined.with_columns(pl.lit("gold").alias("pred_source"))

# Remove as linhas de train que estão em train_refined (usando doc_id como chave)
list_doc_ids = train_refined["doc_id"].to_list()
train_filtered = train.filter(~pl.col("doc_id").is_in(list_doc_ids))

# Mescla os dataframes
train_merged = pl.concat([train_filtered, train_refined], how="vertical")

Vamos treinar o modelo novamente mas dessa vez com os Gold Labels e os pesos diferentes para cada tipo de label

In [ ]:
features = train_merged["review_text"].to_list()
target = train_merged["sentiment"].to_list()
pred_source_np = train_merged["pred_source"].to_numpy()

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
X = model.encode(features, show_progress_bar=True)
y = np.array(target)

Vamos fazer uma validação cruzada estratificada com um Grid Search pra achar o melhor peso para cada tipo de label

In [140]:
clf = XGBClassifier(
    eval_metric="mlogloss", random_state=42
)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

for w_gold in [0.9, 1.0, 1.1]:
    for w_weak in [0.2, 0.3, 0.5, 0.6]:
        sample_weights = np.where(pred_source_np == "gold", w_gold, w_weak)
        score = cross_val_score(
            clf,
            X,
            y_encoded,
            cv=cv,
            scoring="f1_micro",
            params={"sample_weight": sample_weights},
        )
        
        print(f"w_gold: {w_gold}, w_weak: {w_weak}, F1 Score Weighted: {score.mean():.4f}")

w_gold: 0.9, w_weak: 0.2, F1 Score Weighted: 0.8799
w_gold: 0.9, w_weak: 0.3, F1 Score Weighted: 0.8793
w_gold: 0.9, w_weak: 0.5, F1 Score Weighted: 0.8807
w_gold: 0.9, w_weak: 0.6, F1 Score Weighted: 0.8812
w_gold: 1.0, w_weak: 0.2, F1 Score Weighted: 0.8792
w_gold: 1.0, w_weak: 0.3, F1 Score Weighted: 0.8800
w_gold: 1.0, w_weak: 0.5, F1 Score Weighted: 0.8797
w_gold: 1.0, w_weak: 0.6, F1 Score Weighted: 0.8801
w_gold: 1.1, w_weak: 0.2, F1 Score Weighted: 0.8798
w_gold: 1.1, w_weak: 0.3, F1 Score Weighted: 0.8792
w_gold: 1.1, w_weak: 0.5, F1 Score Weighted: 0.8815
w_gold: 1.1, w_weak: 0.6, F1 Score Weighted: 0.8806


> O melhor hiperparâmetro encontrado foi para o peso de gold labels de 1.1 e weak labels de 0.5

In [141]:
sample_weights = np.where(pred_source_np == "gold", 1.1, 0.5)
clf_gold = clf.fit(X, y_encoded, sample_weight=sample_weights)

In [142]:
# Extrai os textos e labels reais do holdout
holdout_texts = holdout["review_text"].to_list()
holdout_y = holdout["sentiment"].to_list()

# Gera embeddings para o holdout
holdout_X = model.encode(holdout_texts, show_progress_bar=True)

# Faz predições
holdout_pred_encoded = clf_gold.predict(holdout_X)
holdout_pred = le.inverse_transform(holdout_pred_encoded)

# Avaliação
print(classification_report(holdout_y, holdout_pred, digits=4))
print("Confusion Matrix:")
print(confusion_matrix(holdout_y, holdout_pred))


Batches: 100%|██████████| 7/7 [00:01<00:00,  5.16it/s]

              precision    recall  f1-score   support

    negativo     0.8148    0.8627    0.8381        51
      neutro     1.0000    0.1034    0.1875        29
    positivo     0.8322    0.9917    0.9049       120

    accuracy                         0.8300       200
   macro avg     0.8823    0.6526    0.6435       200
weighted avg     0.8521    0.8300    0.7839       200

Confusion Matrix:
[[ 44   0   7]
 [  9   3  17]
 [  1   0 119]]


> Conseguimos aumentar a performance do modelo, principalmente o F1 Score dos casos Negativos e dos casos Neutros. Um ponto interessante é a Precision dos casos neutros que atingiu 100% o que nos diz que o modelo não apresentará, provavelmente, falsos positivos para essa classe.

## Salvando o modelo treinado

In [144]:
import pickle

dict_model = {
    "model": clf_gold,
    "label_encoder": le,
    "sentence_transformer": model 
}

model_path = Path().resolve().parent / "model" / "sentiment_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(dict_model, f)